<a href="https://colab.research.google.com/github/troymerales/python-notebooks-nm/blob/main/Linear_Systems_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Linear Systems Solver
In this notebook, I will show you a program that evaluates systems of linear equations using three methods: **Forward Substitution**, **Backward Substitution**, and **Gauss-Seidel**. Note that forward and backward substitutions only deal with lower and upper triangular matrices respectively. Thus, we will introduce another code to derive such matrices, called **LU Decomposition**.

# Forward Substitution

Forward Substitution is a method used to solve a system of the form $Ly=b$, where $L$ is a lower triangular matrix, $y$ is the unknown vector, and $b$ is the known right hand vector.

In [ ]:
import numpy as np

def forsub(L, b):
    n = L.shape[0]
    x = np.zeros(n)
    for i in range(n):
        x[i] = ((b[i] - L[i, :i] @ x[:i]) / L[i, i]).item()
    return x

Say we have the following linear system: \\
$
\begin{array}{r}
y_1=5 \\
2y_1+y_2=8 \\
-y_1+y_2+y_3=4
\end{array}
$ \\
Converting this to a matrix, we have, \\
$ \begin{bmatrix}
1 & 0 & 0
\\ 2 & 1 & 0
\\ -1 & 1 & 1
\end{bmatrix} $
$ \begin{bmatrix}
y_1 \\
y_2 \\
y_3
\end{bmatrix} =
\begin{bmatrix}
5 \\
8 \\
4
\end{bmatrix} $. \\
Note that the matrix is a lower triangular matrix. Thus, we can use forward substitution.


In [ ]:
L=np.array([[1,0,0],
           [2,1,0],
           [-1,1,1]])
b=np.array([[5],
           [8],
           [4]])
print("Solution: ", forsub(L,b))

Solution:  [ 5. -2. 11.]


# Backward Substitution

On the other hand, Forward Substitution is used to solve a system of the form $Ux=d$, where $U$ is an upper triangular matrix, $x$ is the unknown vector, and $d$ is the known right hand vector.

In [ ]:
import numpy as np

def backsub(U, d):
    n = U.shape[0]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = ((d[i] - U[i, i+1:] @ x[i+1:])).item() / U[i, i].item()
    return x

Say we have the following linear system: \\
$
\begin{array}{r}
2x_1+x_2+x_3=5 \\
-8x_2-2x_3=-2 \\
x_3=11
\end{array}
$ \\
Converting this to a matrix, we have, \\
$ \begin{bmatrix}
2 & 1 & 1
\\ 0 & -8 & -2
\\ 0 & 0 & 1
\end{bmatrix} $
$ \begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix} =
\begin{bmatrix}
5 \\
-2 \\
11
\end{bmatrix} $. \\
Note that the matrix is an upper triangular matrix. Thus, we can use backward substitution.


In [ ]:
U=np.array([[2,1,1],
            [0,-8,-2],
            [0,0,1]])
d=np.array([[5],
            [-2],
            [11]])
print("Solution: ", backsub(U,d))

Solution:  [-1.75 -2.5  11.  ]


# LU Decomposition
Forward subsitution is only used for lower triangular matrices or matrices that only have entries on the lower diagonal while the other only has zeros, and backward substitution is used for upper triangular matrices. To be able to solve for any kind of matrix that has a real solution, we decompose said matrix.

In [ ]:
import numpy as np

def LU_decomposition(A):
    m, n = A.shape
    U = np.copy(A).astype(float)
    L = np.eye(m)
    for k in range(n):
        L[k+1:,k] = U[k+1:,k] / U[k,k]
        U[k+1:,k:] -= np.outer(L[k+1:,k], U[k,k:])
    return L, U

Let's try with an example.
Say we have the following linear system: \\
$
\begin{array}{r}
2x_1+x_2+x_3=5 \\
4x_1-6x_2=-2 \\
-2x_1+7x_2+2x_3=9
\end{array}
$ \\
Converting this to a matrix, we have, \\
$ \begin{bmatrix}
2 & 1 & 1
\\ 4 & -6 & 0
\\ -2 & 7 & 2
\end{bmatrix} $
$ \begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix} =
\begin{bmatrix}
5 \\
-2 \\
9
\end{bmatrix} $. \\


In [ ]:
A = np.array([
    [2, 1, 1],
    [4, -6, 0],
    [-2, 7, 2]
])
b=np.array([[5],
            [-2],
            [9]])

Applying the method and simultaneously showing the derived lower and upper matrices,

In [ ]:
L,U=LU_decomposition(A)
print("Lower triangular matrix: \n", L)
print("Upper triangular matrix: \n", U)

Lower triangular matrix: 
 [[ 1.  0.  0.]
 [ 2.  1.  0.]
 [-1. -1.  1.]]
Upper triangular matrix: 
 [[ 2.  1.  1.]
 [ 0. -8. -2.]
 [ 0.  0.  1.]]


Now, the linear system is now of the form $(LU)x=b$. Note that matrix manipulation is associative. We first solve for $Ux=y$ so we're left with $Ly=b$

In [ ]:
y=forsub(L,b)
y

array([  5., -12.,   2.])

Now, we have

In [ ]:
x = backsub(U, y)
print("Solution: ", x)

Solution:  [1. 1. 2.]


# Gauss-Seidel
Gauss-Seidel Method is an iterative algorithm used to solve systems of linear equations, especially when the system is large and sparse.

In [ ]:
import numpy as np

def gaussSeidel(A, b, x0, tol, max_iter):
    n = A.shape[0]
    x = np.copy(x0)
    count_iter = 0
    while np.linalg.norm(A @ x - b) / np.linalg.norm(b) >= tol:
        count_iter += 1
        if count_iter > max_iter:
            print('Method failed after {count_iter} iterations')
            break
        for i in range(n):
            x[i] = (b[i] - A[i, :i] @ x[:i] - A[i, i+1:] @ x[i+1:]) / A[i, i]
    return x, count_iter


Let's try with the following matrix: \\
$
\begin{bmatrix}
10 & -1 & 2 & 0 & 3 \\
1 & 12 & -3 & 2 & -1 \\
2 & -1 & 15 & -4 & 1 \\
0 & 2 & -2 & 20 & -3 \\
3 & -1 & 1 & -2 & 25
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5
\end{bmatrix}
=
\begin{bmatrix}
15 \\
10 \\
25 \\
30 \\
20
\end{bmatrix}
$

In [ ]:
A = np.array ([[10, -1, 2, 0, 3],
              [1, 12, -3, 2, -1],
              [2, -1, 15, -4, 1],
              [0, 2, -2, 20, -3],
              [3, -1, 1, -2, 25]])
b = np.array([[15],
              [10],
              [25],
              [30],
              [20]])

Applying Gauss-Seidel with a tolerance of $10^{-12}$ and max iterations of $10000$,

In [ ]:
x0 = np.zeros((5,1))
x,iter=gaussSeidel(A,b,x0,10**-12,10000)
print("Solution:\n", x)
print("Number of iterations: ",iter)

Solution:
 [[0.96646782]
 [1.03514018]
 [2.01200606]
 [1.71500908]
 [0.78214995]]
Number of iterations:  13
